In [34]:
import openai, json

client = openai.OpenAI()
messages = []

In [35]:
def get_weather(city):
    return '33 degrees Celsius, Sunny'

FUNCTION_MAP = {
    'get_weather': get_weather
}

In [36]:
# tool의 schema
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather for",
                    },
                },
                "required": ["city"],
            },
        }
    }
]

In [37]:
from openai.types.chat import ChatCompletionMessage

def process_ai_response(message: ChatCompletionMessage):
    print(f"[DEBUG] process_ai_response called")  # 디버그 추가
    if message.tool_calls:
        print(f"[DEBUG] Tool calls detected: {len(message.tool_calls)}")  # 디버그 추가
        # ai 가 코드 실행 요청을 한 것을 메시지 메모리에 추가
        messages.append({
            "role": "assistant",
            "content": message.content or "",
            "tool_calls": [{
                "id": tool_call.id,
                "type": "function",  # type 필드 추가
                "function": {
                    "name": tool_call.function.name,
                    "arguments": tool_call.function.arguments
                }
            } for tool_call in message.tool_calls]
        })
    
        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments
            
            print(f"Function Call: {function_name} with arguments {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}
            
            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

            print(f"Ran {function_name} with args {arguments} got result: {result}")
            
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": result
            })
        call_ai()
    else:
        print(f"[DEBUG] No tool calls, printing response")  # 디버그 추가
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI: {message.content}")

def call_ai():
    print(f"[DEBUG] call_ai called")  # 디버그 추가
    # 메시지 memory 와 tool schema 를 AI 에게 전달해 호출
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS
    )
    process_ai_response(response.choices[0].message)


In [ ]:
while True:
    # user가 메세지를 보냄
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        break
    else:
        # user 메세지를 메모리에 추가
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()

User: hi my name is daeyeol
[DEBUG] call_ai called
[DEBUG] process_ai_response called
[DEBUG] No tool calls, printing response
AI: Hello Daeyeol! How can I assist you today?
[DEBUG] process_ai_response called
[DEBUG] No tool calls, printing response
AI: Hello Daeyeol! How can I assist you today?
